<a href="https://colab.research.google.com/github/nbeeeel/Data-Loading/blob/main/Object-Detection/Data_Loading_%26_Cleaning_For_Object_Detection_(YOLO)_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

**we need os for reading/modifying the path and shutil to manipulate the files.XML element tree is for reading the XML files and modifying the values.**

In [ ]:
import os
import xml.etree.ElementTree as ET

In [ ]:
import shutil

#Mounting Drive

**We will mount the google colab to our system's google drive, to access the and store files in our system for later use**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**From google colab we will import files library to upload kaggle.json file that we will download from our kaggle account in order to connect it to our google colab account for direct dataset downloading**

In [ ]:
from google.colab import files
files.upload()

#Setting up api-pin for kaggle dataset-downloading

**In order to download the dataset from kaggle, we need to follow these steps:**

* Login to your kaggle account.
* Click on your profile picture, a menu will slide. There click on the Account icon.
* There scroll down until you find the api section.There click on create new api.
* Download it in your computer and load it by running the above cell with files.upload() method. 
*   Then follow the below procedure to grant permissions and set up the api on google colab.

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

**To download datasets from kaggle you can do one of the following:**
1.   !kaggle datasets download -d **(dataset source from the url i.e. kaggle.com/datasets/"dataset source")**
2.   !kaggle competitions download -c (dataset source from the url i.e. kaggle.com/competitions/"dataset source")

**Remember competitions is for competitions datasets and the first one is for general datasets.You'll clearly see the dataset type in the url.**


In [ ]:
!kaggle datasets download -d alvarole/coffee-leaves-disease

100% 1.39G/1.40G [00:48<00:00, 32.3MB/s]
100% 1.40G/1.40G [00:49<00:00, 30.6MB/s]


In [ ]:
!unzip coffee-leaves-disease.zip

Archive:  coffee-leaves-disease.zip
  inflating: miner_img_xml/bicho_mineiro0.jpg  
  inflating: miner_img_xml/bicho_mineiro0.xml  
  inflating: miner_img_xml/bicho_mineiro1.jpg  
  inflating: miner_img_xml/bicho_mineiro1.xml  
  inflating: miner_img_xml/bicho_mineiro10.jpg  
  inflating: miner_img_xml/bicho_mineiro10.xml  
  inflating: miner_img_xml/bicho_mineiro100.jpg  
  inflating: miner_img_xml/bicho_mineiro100.xml  
  inflating: miner_img_xml/bicho_mineiro101.jpg  
  inflating: miner_img_xml/bicho_mineiro101.xml  
  inflating: miner_img_xml/bicho_mineiro102.jpg  
  inflating: miner_img_xml/bicho_mineiro102.xml  
  inflating: miner_img_xml/bicho_mineiro103.jpg  
  inflating: miner_img_xml/bicho_mineiro103.xml  
  inflating: miner_img_xml/bicho_mineiro104.jpg  
  inflating: miner_img_xml/bicho_mineiro104.xml  
  inflating: miner_img_xml/bicho_mineiro105.jpg  
  inflating: miner_img_xml/bicho_mineiro105.xml  
  inflating: miner_img_xml/bicho_mineiro106.jpg  
  inflating: miner_img_x

#Creating Directories for Organizing Raw Dataset and Storing it into permanently in google drive

In [ ]:
os.chdir('/content/')

**Moving the dataset into the drive by using shutil.move method**

In [ ]:
miner_raw='/content/miner_img_xml/'
rust_raw='/content/rust_xml_image/'

In [ ]:
fdata='/content/drive/MyDrive/Projects/TensorFlow3/Dataset/miner/'
fdata1='/content/drive/MyDrive/Projects/TensorFlow3/Dataset/rust/'

In [ ]:
for files in os.listdir(miner_raw):
  shutil.move(miner_raw+files,fdata)
for files in os.listdir(rust_raw):
  shutil.move(rust_raw+files,fdata1)

In [ ]:
os.chdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/')

In [ ]:
!ls

miner  rust


**Creating new directory structure to store the images and annotations files into the sepearate directories**

In [ ]:
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/images')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/annot')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/images')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/annot')

#Filtering Annotation files 

**Filtering the xml files from the raw dataset and moving them to the annot directory defined in the above cells. The files[-3:] checks for the last 3 alphabets of the string filename and allows only xml files to move via the conditional statement.**

In [ ]:
for files in os.listdir(fdata):
  if files[-3:]=="xml":
    shutil.move(fdata+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/annot/')

In [ ]:
for files in os.listdir(fdata1):
  if files[-3:]=="xml":
    shutil.move(fdata1+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/annot/')

**Now moving the remaining files i.e. to the their corresponding images directories**

In [ ]:
for files in os.listdir(fdata):
  shutil.move(fdata+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/images/')

for files in os.listdir(fdata1):
  shutil.move(fdata1+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/images/')

**Creating the annot_txt directories for each class to put the yolo txt annotation files.**

In [ ]:
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/Annot_txt')
os.mkdir('/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/Annot_txt')

In [ ]:
xml1='/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/annot/'
xml2='/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/annot/'

#Converting the XML annotations to yolo txt annotations.

**The below functions read the xml files and extract the yolo required annotations and classes in the text format. Then the list of the annotation info extracted is mapped on to individual text files and stored in the directories created in the above cells.**

In [ ]:
def parsing_data(fl):
  mytree= ET.parse(fl)
  myroot = mytree.getroot()

  info_dict={}
  info_dict['bboxes']=[]

  #parsing the xml tree
  for elem in myroot:
    #getting the file name
      if elem.tag=="filename":
        info_dict['filename']=elem.text

    #getting the image size
      elif elem.tag=="size":
        image_size=[]
        for subelem in elem:
          image_size.append(int(subelem.text))

        info_dict['image_size']=tuple(image_size)

    #getting the details of the bounding box
      elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict["bboxes"].append(bbox)
          
  return info_dict

In [ ]:
#Dictionary that maps class name to IDs
class_name_to_id_mapping = {"bicho_mineiro":0,"ferrugem":1}

#Convert the info to the required yolo format
def convert_to_yolo(data,path):
  print_buffer=[]

  #for each bounding box
  for b in data["bboxes"]:
    try:
      class_id = class_name_to_id_mapping[b["class"]]
    except KeyError:
      print("Invalid Class. Must be one from", class_name_to_id_mapping.keys())
    #Transform the bounding box coordinates
    b_center_x = (b["xmin"]+b["xmin"])/2
    b_center_y = (b["ymin"]+b["ymax"])/2
    b_width = (b["xmax"] - b["xmin"])
    b_height = (b["ymax"] - b["ymin"])

    #Normalizing the coordinates
    image_w, image_h, image_c = data["image_size"]
    b_center_x /= image_w
    b_center_y /= image_h
    b_width /= image_w
    b_height /= image_h

    #Writing the bounding box details to the file
    print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
  
  #Name the file which we have to save
  save_file_name = os.path.join(path, data["filename"].replace("jpg","txt"))

  #Save the annotation to disk
  print("\n".join(print_buffer), file=open(save_file_name, "w"))

**Pass the path to the xml files for each class to listdir parameters.**

In [ ]:
for filename in os.listdir(xml1):
  data=parsing_data(xml1+filename)
  convert_to_yolo(data,xml1)

In [ ]:
for filename in os.listdir(xml2):
  data=parsing_data(xml2+filename)
  convert_to_yolo(data,xml2)

Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_mineiro', 'ferrugem'])
Invalid Class. Must be one from dict_keys(['bicho_m

**Now we move the text annotation files created in the same directory as the xml annotations to a new directory which we created in the above cell blocks**

In [ ]:
for files in os.listdir(xml1):
  if files[-3:]=="txt":
    shutil.move(xml1+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Miner/Annot_txt/')

In [ ]:
for files in os.listdir(xml2):
  if files[-3:]=="txt":
    shutil.move(xml2+files,'/content/drive/MyDrive/Projects/TensorFlow3/Dataset/Rust/Annot_txt/')

#**END Of the code.**
